<a href="https://colab.research.google.com/github/djelloulbechki/Djelloul_Bechki/blob/master/odoo_ogx_real_estate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Thread 1: Initial Setup Instructions for the Custom Module

Since you’ve already run the Docker Compose code and set up the volume mapping, you’ve completed the most important step: mounting the addons folder.

Look at this line in your `docker-compose.yml`:
```yaml
- ./addons:/mnt/extra-addons
```
This means any folder you create inside the `odoo-stack/addons` directory on your host will be automatically read by Odoo as if it were inside the container.

Here are the practical steps (in exact order) to create your real estate module:

1. **Create the folder structure** (on your server)
   Inside the main `odoo-stack` folder (where `docker-compose.yml` is located), run:

   ```bash
   # Create the addons folder if not already there
   mkdir addons

   # Create the module folder structure
   mkdir -p addons/real_estate_base/models

   # Create empty required files
   touch addons/real_estate_base/__init__.py
   touch addons/real_estate_base/__manifest__.py
   touch addons/real_estate_base/models/__init__.py
   touch addons/real_estate_base/models/property.py
   ```

2. **Write the code** (use nano, vim, or any editor)

   **A. Module manifest: `addons/real_estate_base/__manifest__.py`**
   ```python
   {
       'name': 'Real Estate Base',
       'version': '1.0',
       'category': 'Real Estate',
       'summary': 'Manage Properties, Units, and Owners',
       'depends': ['base', 'crm'],  # Depends on CRM module
       'data': [],
       'installable': True,
       'application': True,
   }
   ```

   **B. Property model: `addons/real_estate_base/models/property.py`**
   ```python
   from odoo import models, fields

   class RealEstateProperty(models.Model):
       _name = 'real_estate.property'
       _description = 'Property'

       name = fields.Char(string="Property Name", required=True)
       description = fields.Text(string="Description")
       location = fields.Char(string="Location")
       price = fields.Float(string="Price")
       bedrooms = fields.Integer(string="Bedrooms")
       active = fields.Boolean(default=True)
       state = fields.Selection([
           ('available', 'Available'),
           ('rented', 'Rented'),
           ('sold', 'Sold')
       ], string="Status", default='available')

       # Link to owner (res.partner = contacts)
       owner_id = fields.Many2one('res.partner', string="Owner")
   ```

   **C. Link everything with `__init__.py` files**

   - `addons/real_estate_base/models/__init__.py`:
     ```python
     from . import property
     ```

   - `addons/real_estate_base/__init__.py`:
     ```python
     from . import models
     ```

3. **Activate the module in Odoo UI** (localhost:8069 or your VPS IP:8069)

   1. Enable Developer Mode (Settings → Activate the developer mode)
   2. Go to Apps menu
   3. Click **Update Apps List** (very important — refreshes custom addons)
   4. Search for "Real Estate Base" and click **Activate**

**What happens in PostgreSQL?**
Once you click Activate, Odoo connects to the `odoo_db` container and creates a new table named `real_estate_property`.
You can verify it:

```bash
docker exec -it odoo_db psql -U odoo -d OGX -c "\dt real_estate_property"
```

(Use your actual database name — see below if unsure.)

### Thread 2: Module Not Appearing After Update Apps List (Common Fix)

Module not showing after **Update Apps List** is a very common issue in Odoo. It’s usually one of these three causes:

1. **Wrong folder path** (most important)
   Make sure the structure is exactly:
   ```
   odoo-stack/addons/real_estate_base/__manifest__.py
   ```
   Run this to verify:
   ```bash
   ls -R ~/odoo-stack/addons/real_estate_base
   ```

2. **Search filter hiding custom modules**
   - Go to Apps
   - In the search bar, click the **(x)** next to the word "Apps" to remove the default filter
   - Then search "Real Estate"

3. **Syntax error in `__manifest__.py`**
   Odoo silently ignores the whole module if there’s a missing comma, bracket, etc. Double-check it starts with `{` and ends with `}`.

4. **Quick restart trick**
   Sometimes Odoo needs a full refresh:
   ```bash
   cd ~/odoo-stack
   docker compose restart odoo
   ```
   Wait 10 seconds, then Update Apps List again.

5. **Check logs for errors**
   If still not visible:
   ```bash
   docker logs odoo_app --tail 50
   ```
   Look for "Critical" or "Error" related to `real_estate_base` — copy and share it if you see any.

**Quick win**: Try removing the "Apps" filter first (step 2) — it’s the most common reason!

### Thread 3: Table Not Created in Database After Activate

If you activated the module but the table still doesn’t appear in PostgreSQL, it means Odoo read the folder but skipped executing the model code (empty install).

Most likely causes:

1. **Missing imports in `__init__.py` files** (very common)
   Odoo won’t load models automatically.

   Make sure:

   - `addons/real_estate_base/__init__.py`:
     ```python
     from . import models
     ```

   - `addons/real_estate_base/models/__init__.py`:
     ```python
     from . import property
     ```

2. **Need to Upgrade the module**
   If you activated before finishing the code or `__init__.py` files, Odoo marked it as installed and won’t re-read unless you upgrade.

   Steps:
   1. Go to Apps
   2. Search "Real Estate Base"
   3. Click the three dots (or arrow) → **Upgrade**

3. **Wrong database name**
   You named your database **OGX** (not the default `postgres`).
   Always use the correct DB name when checking:

   ```bash
   docker exec -it odoo_db psql -U odoo -d OGX -c "\dt real_estate_property"
   ```

   To list all databases:
   ```bash
   docker exec -it odoo_db psql -U odoo -l
   ```

4. **Final check: logs after upgrade**
   After upgrading, check logs for errors:
   ```bash
   docker logs odoo_app --tail 100 | grep real_estate
   ```

**Pro tip**: Odoo uses the `postgres` container only as a gateway. Real data lives in the database you created (in your case: **OGX**).

Did the table appear after using `-d OGX` and upgrading? Let me know the result! 🚀